# ORTHOGONAL POLYNOMIAL DENSITY ESTIMATION
## Preliminaries
### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
import pandas as pd
from uuid import uuid4

from lpde.geometry import WidthOf, Window, PointAt, BoundingBox, Mapper
from lpde.estimator import SerialEstimate
from lpde.estimator.datatypes import Event, Degree, Action, Scalings

### Notebook settings

In [2]:
%matplotlib notebook

## Density Estimation
### Initialize

In [4]:
legendre_width = WidthOf(1.8)

center = PointAt(0, 0)
window = Window(1.8, 1.8)
bounds = BoundingBox(center, window)

mapper = Mapper(bounds, legendre_width)

degree = Degree(20, 20)
density = SerialEstimate(degree, mapper)

action = Action.ADD
point = PointAt(0.5, 0.5)
event = Event(uuid4(), action, point)

### Create mock data streams

In [5]:
def gaussian():
    x, y = np.random.multivariate_normal((0,0), ((0.1,0), (0,0.1)))
    if (-0.9 <= x <= 0.9) and (-0.9 <= y <= 0.9):
        return x, y
    else:
        return gaussian()

def uniform():
    return np.random.uniform(low=-0.9, high=0.9, size=2)

locations = []

def new_event(dist):
    location = dist()
    locations.append(location)
    point = PointAt(*location)
    return Event(uuid4(), Action(1), point)

def random_event(dist):
    event_type = np.random.randint(low=-1, high=2)
    if event_type == 1:
        location = dist()
        point = PointAt(*location)
        return Event(uuid4(), Action(1), point)
    elif event_type == 0:
        location = dist()
        point = PointAt(*location)
        column = density._phi.sample(1, axis=1).columns.values[0]
        return Event(column, Action(0), point)
    column = density._phi.sample(1, axis=1).columns.values[0]
    return Event(column, Action(-1))

### Timings of density estimation

In [6]:
%%time
for i in range(1000):
    density.update_with(new_event(uniform))

/Users/georg/Documents/Python/LPDE/lpde/estimator/estimator.py:90: RuntimeWarning: overflow encountered in double_scalars
  return self.__neg_log_l(c[1:]) + c[0]*self.__norm(c[1:])
/Users/georg/Documents/Python/LPDE/lpde/estimator/estimator.py:98: RuntimeWarning: overflow encountered in square
  return -log(square(c.dot(self.__phi_ijn.values))).sum()
/Users/georg/Documents/Python/LPDE/lpde/estimator/estimator.py:94: RuntimeWarning: overflow encountered in multiply
  self.__grad_c[1:] = self.__grad_neg_log_l(c[1:]) + 2.0*c[0]*c[1:]


CPU times: user 1min 7s, sys: 5.6 s, total: 1min 12s
Wall time: 42 s


In [7]:
%%time
for i in range(1000):
    density.update_with(random_event(gaussian))

CPU times: user 1min 11s, sys: 9.28 s, total: 1min 21s
Wall time: 40.6 s


In [8]:
%%time
for i in range(1000):
    density.at(new_event(uniform).location)

CPU times: user 629 ms, sys: 10.7 ms, total: 640 ms
Wall time: 465 ms


__Timings__

33 ms per additive update with uniform distribution

31 ms per random update with uniform distribution

0.6 ms per evaluation at point

### Plot final density

In [ ]:
x_grid = np.linspace(-0.9, 0.90, 100)
y_grid = np.linspace(-0.9, 0.90, 100)
x_grid, y_grid = np.meshgrid(x_grid, y_grid)

p_hat = density._on(x_grid, y_grid)

fig, ax = plt.subplots()
ax.set(xlabel=r'$x$', ylabel=r'$y$')
ax.scatter(*zip(*locations), s=5, c='k')
contour = ax.contourf(x_grid, y_grid, p_hat, 9, cmap='viridis', alpha=0.9)
cbar = plt.colorbar(contour, ax=ax, label=r'$p(x)$')
fig.tight_layout()

## Coefficient smoothing

In [ ]:
coeffs = []

for i in range(1000):
    density.update_with(new_event(gaussian))
    coeffs.append(density._c.tolist())  

coeffs = np.array(coeffs).T

In [ ]:
ij = 100

fig, ax = plt.subplots()
ax.plot(coeffs[ij])

In [ ]:
import scipy.signal as scps

In [ ]:
b, a = scps.butter(2, 0.03)
plt.plot(scps.lfilter(b, a, coeffs[ij]))

In [ ]:
class A:
    def __private(self):
        print('Privat!')

class B:
    def __init__(self, other):
        print(other.__class__.__name__)
        self.__method = getattr(other, '_A__private')
        
    def knock(self):
        self.__method()

In [ ]:
a = A()

In [ ]:
b = B(a)

In [ ]:
b.knock()

In [ ]:
a = 3

In [ ]:
b = a

In [ ]:
a = 2

In [ ]:
b